## 設定環境

* 執行階段 > 變更執行階段類型 > 使用GPU
* 安裝Pop2Piano

In [ ]:
# 下載安裝套件
!apt-get install -y fluidsynth

# Fix
!git clone https://github.com/taipingeric/pop2piano
# Official
# !git clone https://github.com/sweetcocoa/pop2piano/

!cd pop2piano
!pip install pretty-midi==0.2.9 omegaconf==2.1.1 youtube-dl==2021.12.17 transformers==4.16.1 pytorch-lightning essentia==2.1b6.dev609 note-seq==0.0.3 pyFluidSynth==1.3.0
# 下載官方模型參數
!wget https://github.com/sweetcocoa/pop2piano/releases/download/dpi_2k_epoch/model-1999-val_0.67311615.ckpt

## Import 套件

In [ ]:
import os 
import sys
sys.path.append("pop2piano")
import glob
import random

import torch
import torchaudio
import librosa
import numpy as np
import pandas as pd
import IPython.display as ipd
import soundfile as sf
from google.colab import files

from tqdm.auto import tqdm
from omegaconf import OmegaConf
import note_seq

from utils.dsp import get_stereo
from utils.demo import download_youtube
from transformer_wrapper import TransformerWrapper
from midi_tokenizer import MidiTokenizer, extrapolate_beat_times
from preprocess.beat_quantizer import extract_rhythm, interpolate_beat_times

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
config = OmegaConf.load("pop2piano/config.yaml")
wrapper = TransformerWrapper(config)
wrapper = wrapper.load_from_checkpoint("model-1999-val_0.67311615.ckpt", config=config).to(device)
model = "dpipqxiy"
wrapper.eval()
""

## 選擇編曲者1~21

In [ ]:
composer = "composer1"

## 上傳歌曲檔案 (.wav, .mp3)

In [ ]:
file = files.upload()
audio_file = list(file.keys())[0]

In [ ]:
pm, composer, mix_path, midi_path = wrapper.generate(
    audio_path=audio_file, 
    composer=composer, 
    model=model,
    show_plot=True, 
    save_midi=True, 
    save_mix=True, 
)